In [13]:
from struct import unpack
from pathlib import Path
from enum import Enum

import json
import xmltodict
import numpy as np

import matplotlib.pyplot as plt

from astropy.time import Time
import astropy.units as u
from astropy.table import Table

from timdimm_tng.scheduler import Sequence, Observation, Schedule, make_full_schedule

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [8]:
stars = Table.read("/home/tim/MMT/timdimm_tng/src/timdimm_tng/star_list.ecsv")
stars

HR,Name,RA2000,Dec2000,Vmag,B-V,SED,SpType,Duplicity,Coordinates
,,,,,,,,,"deg,deg"
int64,str14,str8,str9,float64,float64,str3,str12,str15,SkyCoord
98,Bet Hyi,0:25:45,-77:15:15,2.8,0.62,G05,G2IV,--,"6.437793155,-77.25424612"
99,Ankaa,0:26:17,-42:18:22,2.39,1.09,K03,K0III,"0.1"" dm=0.0","6.571047515,-42.305987194"
188,Diphda,0:43:35,-17:59:12,2.04,1.02,K03,G9.5IIICH-1,--,"10.897378736,-17.986606316"
472,Achernar,1:37:43,-57:14:12,0.46,-0.16,B55,B3Vpe,--,"24.428522833,-57.236752806"
591,Alp Hyi,1:58:46,-61:34:11,2.86,0.28,F05,F0V,--,"29.69224409758834,-61.56982215319751"
1790,Bellatrix,5:25:08,+06:20:59,1.64,-0.22,B05,B2III,"179"" dm=10.5","81.282763557,6.349703264"
1865,Arneb,5:32:44,-17:49:20,2.58,0.21,F05,F0Ib,"AB 36"" dm=8.5","83.18256716167,-17.82228927222"
1903,Alnilam,5:36:13,-01:12:07,1.7,-0.19,B05,B0Ia,"180"" dm=8.7","84.053388941,-1.201919136"


In [14]:
make_full_schedule()

{'SchedulerList': {'@version': '1.5', 'Profile': 'timdimm_sim', 'Job': [{'Name': 'Bet Hyi', 'Priority': '18', 'Coordinates': {'J2000RA': '6.437793155', 'J2000DE': '-77.25424612'}, 'PositionAngle': None, 'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq', 'StartupCondition': {'Condition': 'ASAP'}, 'Constraints': {'Constraint': [{'@value': '30', '#text': 'MinimumAltitude'}, {'@value': '20', '#text': 'MoonSeparation'}, 'EnforceWeather', 'EnforceTwilight']}, 'CompletionCondition': {'Condition': 'Loop'}, 'Steps': {'Step': ['Track', 'Align']}}, {'Name': 'Ankaa', 'Priority': '16', 'Coordinates': {'J2000RA': '6.571047515', 'J2000DE': '-42.305987194'}, 'PositionAngle': None, 'Sequence': '/home/tim/MMT/timdimm_tng/src/timdimm_tng/templates/timdimm_sequence.esq', 'StartupCondition': {'Condition': 'ASAP'}, 'Constraints': {'Constraint': [{'@value': '30', '#text': 'MinimumAltitude'}, {'@value': '20', '#text': 'MoonSeparation'}, 'EnforceWeather', 'EnforceTwilight']